# Trabajo final DMC

## 1. Recolección de datos con la API de Spotify

### Configuración de la API

In [2]:
# Librerías requeridas

## Spotipy
import spotipy
from spotipy.oauth2 import SpotifyOAuth

## Generales
from pprint import pprint

Inicialmente se debe configurar una aplicación en el *dashboard* de Spotify, según el [Web API tutorial](https://developer.spotify.com/documentation/web-api/quick-start/):

1. Ir a [developer.spotify.com](https://developer.spotify.com/)
2. Ir a la pestaña *DASHBOARD*
3. Ingresar con el nombre de usuario y contraseña de Spotify
4. Crear una nueva aplicación (*CREATE AN APP*), definiendo un nombre y agregando una descripción
5. *EDIT SETTINGS* -> Redirect URIs -> http://localhost:8080. Requerido para la autenticación desde Python
6. *USERS AND ACCESS* -> *ADD NEW USER* -> Definir nombre y correo electrónico de la cuenta de Spotify. Este será el usuario que podrá acceder a la API desde Python


### Conexión spotipy-app

In [3]:
cid = '923e41f1eb3f4f638b102398bb2bee6b'
secret = 'fc0ad2c973de49b99ce16d84e86f0f14'
redirect_uri = 'http://localhost:8080'
scope = 'user-top-read'

client_credentials_manager = SpotifyOAuth(client_id=cid, client_secret=secret, redirect_uri=redirect_uri, scope=scope)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=30)

In [4]:
# Mostrar el usuario actual
me = sp.me()
username = me['id']

pprint(me)
print('-'*10)
print('username:', username)

{'display_name': 'Jack Cruz',
 'external_urls': {'spotify': 'https://open.spotify.com/user/31prj7pij5zeghgoai4bws4bxshm'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/users/31prj7pij5zeghgoai4bws4bxshm',
 'id': '31prj7pij5zeghgoai4bws4bxshm',
 'images': [{'height': None,
             'url': 'https://platform-lookaside.fbsbx.com/platform/profilepic/?asid=536674957764087&height=300&width=300&ext=1685156019&hash=AeSJzhJjHl30f7cNy2w',
             'width': None}],
 'type': 'user',
 'uri': 'spotify:user:31prj7pij5zeghgoai4bws4bxshm'}
----------
username: 31prj7pij5zeghgoai4bws4bxshm


In [5]:
# Testear la conexión con Spotipy enviando una petición search
artist = 'Gian Marco'
results = sp.search(q='artist:'+artist, type='artist')
pprint(results)

{'artists': {'href': 'https://api.spotify.com/v1/search?query=artist%3AGian+Marco&type=artist&offset=0&limit=10',
             'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2gDqGAadPIPiA7LtmNn74g'},
                        'followers': {'href': None, 'total': 1260844},
                        'genres': ['peruvian rock'],
                        'href': 'https://api.spotify.com/v1/artists/2gDqGAadPIPiA7LtmNn74g',
                        'id': '2gDqGAadPIPiA7LtmNn74g',
                        'images': [{'height': 640,
                                    'url': 'https://i.scdn.co/image/ab6761610000e5eb4ea81c45c3e7f96244ece553',
                                    'width': 640},
                                   {'height': 320,
                                    'url': 'https://i.scdn.co/image/ab676161000051744ea81c45c3e7f96244ece553',
                                    'width': 320},
                                   {'height': 160,
                        

### Listado de top-20 tracks                              

Como punto de partida se usarán los [user's top tracks](https://developer.spotify.com/documentation/web-api/reference/#endpoint-get-users-top-artists-and-tracks) que requiere privilegios tipo `user-top-read`.

Otros puntos de partida sugeridos:
- `sp.user_current_followed_artists`
- `sp.current_user_playlists`
- `sp.current_user_recently_played`
- `sp.current_user_top_artists`

entre otros.

In [4]:
# top20 = sp.current_user_recently_played()

toptracks = []
results = sp.current_user_top_tracks(time_range='short_term')
toptracks.extend(results['items'])

while results['next'] and len(toptracks) < 100:
    results = sp.next(results)
    toptracks.extend(results['items'])

# El resultado está almacenado en un diccionario en donde 'items'
# contiene la información de los tracks

for i, item in enumerate(toptracks):
    print(i, '. ', item['name'], '//', item['artists'][0]['name'], '//', item['preview_url'])

0 .  Pedindo Esmolas // Jésed // https://p.scdn.co/mp3-preview/794c7278f648fa0400b9f8daa3dbc765dc750150?cid=923e41f1eb3f4f638b102398bb2bee6b
1 .  Volví // Aventura // https://p.scdn.co/mp3-preview/4157475ff15a2340fcf5746fb7cdcb6dee7490ea?cid=923e41f1eb3f4f638b102398bb2bee6b
2 .  Promise (feat. Usher) // Romeo Santos // https://p.scdn.co/mp3-preview/60f8555a10c67bb6801f9ea842ba5d262e3b6645?cid=923e41f1eb3f4f638b102398bb2bee6b
3 .  De Ellos Aprendí // David Rees // https://p.scdn.co/mp3-preview/bdfebffb548a4897cf22d93bb30e565c7edd8216?cid=923e41f1eb3f4f638b102398bb2bee6b
4 .  Me Voy Enamorando // Chino & Nacho // None
5 .  Looking For Sade // Peabo Bryson // None
6 .  Into the Desert // Ely Guerra // https://p.scdn.co/mp3-preview/d543349f1e0798ab2e5c859fbce31d38920f605d?cid=923e41f1eb3f4f638b102398bb2bee6b
7 .  Gracias a Dios // Mach & Daddy // https://p.scdn.co/mp3-preview/6397e6686cdfd06078b21b1efacae5c63a3f512e?cid=923e41f1eb3f4f638b102398bb2bee6b
8 .  Higher Love // Kygo // https://p

#### Dataset del preferencias del usuario

Con estos tracks crearemos un dataset (en Pandas) que contendrá, por cada pista, su identificador y sus [características sonoras]():

In [5]:
import pandas as pd

# Extraer ids y nombres de las canciones, y extraer audio_features
track_ids = []
track_names = []
features = []

for track in toptracks:
    track_id = track['id']
    track_name = track['name']
    audio_features = sp.audio_features(track_id)
    
    track_ids.append(track_id)
    track_names.append(track_name)
    features.append(audio_features[0])
    
toptracks_df = pd.DataFrame(features, index=track_names)
toptracks_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
Pedindo Esmolas,0.779,0.287,2,-14.267,0,0.0342,0.8120,0.000047,0.1700,0.697,117.982,audio_features,2ETyUK0F39pIYkkoQu6lRu,spotify:track:2ETyUK0F39pIYkkoQu6lRu,https://api.spotify.com/v1/tracks/2ETyUK0F39pI...,https://api.spotify.com/v1/audio-analysis/2ETy...,220721,3
Volví,0.725,0.860,1,-3.726,1,0.1820,0.4220,0.000284,0.0674,0.791,175.896,audio_features,2vmfvSoZBFAt9hhRoEByLi,spotify:track:2vmfvSoZBFAt9hhRoEByLi,https://api.spotify.com/v1/tracks/2vmfvSoZBFAt...,https://api.spotify.com/v1/audio-analysis/2vmf...,230125,4
Promise (feat. Usher),0.764,0.739,1,-6.422,1,0.0574,0.4030,0.000000,0.0821,0.770,132.995,audio_features,0jr6tT2vc4cIFPHb6wufG3,spotify:track:0jr6tT2vc4cIFPHb6wufG3,https://api.spotify.com/v1/tracks/0jr6tT2vc4cI...,https://api.spotify.com/v1/audio-analysis/0jr6...,252253,4
De Ellos Aprendí,0.720,0.572,7,-6.479,1,0.1160,0.1450,0.000000,0.4390,0.675,166.843,audio_features,1DnQiHRa2XnfwIf8qVl5C1,spotify:track:1DnQiHRa2XnfwIf8qVl5C1,https://api.spotify.com/v1/tracks/1DnQiHRa2Xnf...,https://api.spotify.com/v1/audio-analysis/1DnQ...,266076,4
Me Voy Enamorando,0.685,0.912,10,-3.411,1,0.0687,0.0538,0.000000,0.3920,0.511,99.957,audio_features,39Gb80bu5xF0rQtAxTxUS1,spotify:track:39Gb80bu5xF0rQtAxTxUS1,https://api.spotify.com/v1/tracks/39Gb80bu5xF0...,https://api.spotify.com/v1/audio-analysis/39Gb...,235720,4


In [6]:
# Reorganizar columnas
toptracks_df = toptracks_df[["id", "acousticness", "danceability", "duration_ms", "energy", "instrumentalness",  "key", "liveness", "loudness", "mode", "speechiness", "tempo", "valence"]]
toptracks_df.head()

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
Pedindo Esmolas,2ETyUK0F39pIYkkoQu6lRu,0.8120,0.779,220721,0.287,0.000047,2,0.1700,-14.267,0,0.0342,117.982,0.697
Volví,2vmfvSoZBFAt9hhRoEByLi,0.4220,0.725,230125,0.860,0.000284,1,0.0674,-3.726,1,0.1820,175.896,0.791
Promise (feat. Usher),0jr6tT2vc4cIFPHb6wufG3,0.4030,0.764,252253,0.739,0.000000,1,0.0821,-6.422,1,0.0574,132.995,0.770
De Ellos Aprendí,1DnQiHRa2XnfwIf8qVl5C1,0.1450,0.720,266076,0.572,0.000000,7,0.4390,-6.479,1,0.1160,166.843,0.675
Me Voy Enamorando,39Gb80bu5xF0rQtAxTxUS1,0.0538,0.685,235720,0.912,0.000000,10,0.3920,-3.411,1,0.0687,99.957,0.511


#### Tracks candidatas

Partiendo de los top-20 tracks:
1. Extraeremos los artistas correspondientes (sin repeticiones)
2. Se usará la API para ampliar este listado a:
    - Artistas relacionados (`sp.artist_related_artists`)

Con este listado más amplio de artistas, obtenido en (1) y (2) se obtendrá:

3. El listado de albums
4. Y por cada album el listado de tracks

Y finalmente, para el listado resultante se obtendrán las *audio features*. El dataset resultante serán las pistas candidatas, que luego se llevarán al sistema de recomendación

In [7]:
# 1. Extraer los artistas correspondientes a las top-20 tracks (sin repeticiones)
ids_artists = []
print('Artistas de mi top:')
print('=====================')
for item in toptracks:
    artist_id = item['artists'][0]['id']
    artist_name = item['artists'][0]['name']
    print(f'{artist_id}: {artist_name}')
    ids_artists.append(artist_id)

# Depurar lista para evitar repeticiones
ids_artists = list(set(ids_artists))
print(f'Número de artistas (sin repeticiones): {len(ids_artists)}')

Artistas de mi top:
3TXp4wMgvRheVQPf1PEV2P: Jésed
1qto4hHid1P71emI6Fd8xi: Aventura
5lwmRuXgjX8xIwlnauTZIP: Romeo Santos
2CQHyfluB4mliyv193Qn2L: David Rees
5NS0854TqZQVoRmJKSWtFZ: Chino & Nacho
49iKbKGqgn8OESkW5WduX0: Peabo Bryson
1ne2c2YEgt4MmJCJGCsfsZ: Ely Guerra
5Q1vijNjTVhQBoVrZVXms8: Mach & Daddy
23fqKkggKUBHNkbKtXEls4: Kygo
6akDfyocmsREgR5eUXZt3I: Bely Basarte
2W8yFh0Ga6Yf3jiayVxwkE: Dove Cameron
6X8uzem5Dbl4l2so4CeexD: Nil Moliner
3nlHsNqwCSvT988ZfSW1Yh: Ronan Keating
4GNC7GD6oZMSxPGyXy4MNB: Lewis Capaldi
2xobqRIT4uGkrZVPJjRQeY: Mauricio Alen
60nua3AsVSfADZtg5Hdz3W: La Mosca Tse-Tse
4dpARuHxo51G3z768sgnrY: Adele
2MLHBMApNE5h8wIufiTPs7: Duncan Dhu
2urF8dgLVfDjunO0pcHUEe: Alvaro Soler
4TK1gDgb7QKoPFlzRrBRgR: Los Enanitos Verdes
7okwEbXzyT2VffBmyQBWLz: Maná
60uh2KYYSCqAgJNxcU4DA0: Hombres G
39BvzssARgDTZ1Kf0uqNfj: Té de Brujas
4TK1gDgb7QKoPFlzRrBRgR: Los Enanitos Verdes
13wFTN72PGSUxzEHJP5Ljs: Los Caligaris
7An4yvF7hDYDolN4m5zKBp: Soda Stereo
5VQCk9RiLwri99OgOT34kq: Vilma Palma e Va

In [8]:
# 2.1 Ampliar al listado anterior a artistas relacionados
# Por cada artista que sigo y que está en mi top20 buscar artistas similares y añadirlos al listado
print('')
print('Artistas similares:')
print('=====================')
ids_similar_artists = []
for artist_id in ids_artists:
    artists = sp.artist_related_artists(artist_id)['artists']
    for item in artists:
        artist_id = item['id']
        artist_name = item['name']
        print(f'{artist_id}: {artist_name}')
        ids_similar_artists.append(artist_id)

ids_artists.extend(ids_similar_artists)

# Depurar lista para evitar repeticiones
ids_artists = list(set(ids_artists))
print(f'Número de artistas (sin repeticiones): {len(ids_artists)}')


Artistas similares:
4nNwfeVaNJlfz8RdCT5MJO: Gareth Gates
3ZcbVcd3fsf9qKK02UVzGB: Gary Barlow
6X9aYHnQ75YI8o08aoa0iS: Boyzone
2U6gqwyl9F33YxawnFrZG7: Will Young
3NRFinRTEqUCfaTTZmk8ek: Savage Garden
11hIqBsGRPztdjBHCSLClX: Daniel Bedingfield
5lPsVvHVDr6R5mDxRUXdOs: A1
1XgFuvRd7r5g0h844A5ZUQ: Take That
5Z1CCuBsyhEHngq3U5IraY: Westlife
6H4lieipng8aGu3Hbd1UeJ: Shayne Ward
5ny07v4RRsvdv1uQVplMry: Brian McFadden
6rEzedK7cKWjeQWdAYvWVG: Five
3P6nMEGXn0hzBlMndKfLQH: Simon Webbe
6JMHws5haIO6V35YNYDnDw: Atomic Kitten
1kM5rgJvkiDMOoKX56H6pX: S Club 7
2yEkZBBjhzKzt6LF5XMaFi: Blue
3H7pODXycF821OnRyAw2VT: 911
2HcwFjNelS49kFbfvMxQYw: Robbie Williams
6edGSAX5dVpeJVwu1Q0NwJ: Lighthouse Family
7rftfGIYEeZ79sLb58ZBDi: Gabrielle
4TK1gDgb7QKoPFlzRrBRgR: Los Enanitos Verdes
1GImnM7WYVp95431ypofy9: Caifanes
2Ax9wZpdlg4r2zkc3pcI8U: La Unión
3SqzxvGCKGJ9PYKXXPwjQS: Fobia
02Nbktg6lCJiazPM6YYTMz: Miguel Mateos
5oYHL2SijkMY52gKIhYJNb: Elefante
1ZVoRDO29AlDXiMkRLMZSK: La Ley
3HgZDevp7GspkLUAa5cKne: El Tri
4uqzzJg

In [9]:
# Obtener el listado de albums de cada uno de los anteriores artistas.
# Se limitará a únicamente 1 album (limit=1), para evitar tener un listado gigantesco

id_albums = []
nartists = len(ids_artists)
for i, id_artist in enumerate(ids_artists):
    print(f'Procesando artista {i+1} de {nartists}...')
    albums = sp.artist_albums(id_artist, country='PE', limit=1) # para evitar tener una lista gigantesca
    for album in albums['items']:
        id_albums.append(album['id'])
print('¡Listo!')

Procesando artista 1 de 745...
Procesando artista 2 de 745...
Procesando artista 3 de 745...
Procesando artista 4 de 745...
Procesando artista 5 de 745...
Procesando artista 6 de 745...
Procesando artista 7 de 745...
Procesando artista 8 de 745...
Procesando artista 9 de 745...
Procesando artista 10 de 745...
Procesando artista 11 de 745...
Procesando artista 12 de 745...
Procesando artista 13 de 745...
Procesando artista 14 de 745...
Procesando artista 15 de 745...
Procesando artista 16 de 745...
Procesando artista 17 de 745...
Procesando artista 18 de 745...
Procesando artista 19 de 745...
Procesando artista 20 de 745...
Procesando artista 21 de 745...
Procesando artista 22 de 745...
Procesando artista 23 de 745...
Procesando artista 24 de 745...
Procesando artista 25 de 745...
Procesando artista 26 de 745...
Procesando artista 27 de 745...
Procesando artista 28 de 745...
Procesando artista 29 de 745...
Procesando artista 30 de 745...
Procesando artista 31 de 745...
Procesando artist

In [10]:
# Por cada album extraer 3 tracks

id_tracks = []
nalbums = len(id_albums)
for i, id_album in enumerate(id_albums):
    print(f'Procesando album {i+1} de {nalbums}...')
    album_tracks = sp.album_tracks(id_album, limit=1)
    for track in album_tracks['items']:
        id_tracks.append(track['id'])
print(f'¡Listo! Número total de tracks pre-candidatos: {len(id_tracks)}')

Procesando album 1 de 745...
Procesando album 2 de 745...
Procesando album 3 de 745...
Procesando album 4 de 745...
Procesando album 5 de 745...
Procesando album 6 de 745...
Procesando album 7 de 745...
Procesando album 8 de 745...
Procesando album 9 de 745...
Procesando album 10 de 745...
Procesando album 11 de 745...
Procesando album 12 de 745...
Procesando album 13 de 745...
Procesando album 14 de 745...
Procesando album 15 de 745...
Procesando album 16 de 745...
Procesando album 17 de 745...
Procesando album 18 de 745...
Procesando album 19 de 745...
Procesando album 20 de 745...
Procesando album 21 de 745...
Procesando album 22 de 745...
Procesando album 23 de 745...
Procesando album 24 de 745...
Procesando album 25 de 745...
Procesando album 26 de 745...
Procesando album 27 de 745...
Procesando album 28 de 745...
Procesando album 29 de 745...
Procesando album 30 de 745...
Procesando album 31 de 745...
Procesando album 32 de 745...
Procesando album 33 de 745...
Procesando album 34

In [11]:
# En total se tendrán 543 (890) pistas candidatas. Para cada una se extraerán los
# audio_features y todo se almacenará en un dataframe

track_names = []
features = []
ntracks = len(id_tracks)
for i, track_id in enumerate(id_tracks):
    print(f'Procesando track {i+1} de {ntracks}...')
    track_name = sp.track(track_id)['name']
    audio_features = sp.audio_features(track_id)
    
    #No incluir pistas sin "features"
    if audio_features[0] != None:
        track_names.append(track_name)
        features.append(audio_features[0])
print('¡Listo!')

candidatos_df = pd.DataFrame(features,index = track_names)

Procesando track 1 de 745...
Procesando track 2 de 745...
Procesando track 3 de 745...
Procesando track 4 de 745...
Procesando track 5 de 745...
Procesando track 6 de 745...
Procesando track 7 de 745...
Procesando track 8 de 745...
Procesando track 9 de 745...
Procesando track 10 de 745...
Procesando track 11 de 745...
Procesando track 12 de 745...
Procesando track 13 de 745...
Procesando track 14 de 745...
Procesando track 15 de 745...
Procesando track 16 de 745...
Procesando track 17 de 745...
Procesando track 18 de 745...
Procesando track 19 de 745...
Procesando track 20 de 745...
Procesando track 21 de 745...
Procesando track 22 de 745...
Procesando track 23 de 745...
Procesando track 24 de 745...
Procesando track 25 de 745...
Procesando track 26 de 745...
Procesando track 27 de 745...
Procesando track 28 de 745...
Procesando track 29 de 745...
Procesando track 30 de 745...
Procesando track 31 de 745...
Procesando track 32 de 745...
Procesando track 33 de 745...
Procesando track 34

In [12]:
candidatos_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
Aguije,0.467,0.871,0,-5.292,1,0.2180,0.2080,0.000000,0.456,0.605,150.013,audio_features,5jMW9bjIpyziEv5mu5Next,spotify:track:5jMW9bjIpyziEv5mu5Next,https://api.spotify.com/v1/tracks/5jMW9bjIpyzi...,https://api.spotify.com/v1/audio-analysis/5jMW...,191332,3
La Distancia,0.685,0.687,0,-4.580,1,0.0370,0.0876,0.000006,0.078,0.949,159.993,audio_features,2MdaSGTSLkY2ZEVYTEFjAl,spotify:track:2MdaSGTSLkY2ZEVYTEFjAl,https://api.spotify.com/v1/tracks/2MdaSGTSLkY2...,https://api.spotify.com/v1/audio-analysis/2Mda...,228548,4
La esquina de Rowland,0.694,0.780,0,-4.573,1,0.0329,0.1530,0.000000,0.147,0.635,129.919,audio_features,6MEtlpTuzFJQfa8DvZ6N4V,spotify:track:6MEtlpTuzFJQfa8DvZ6N4V,https://api.spotify.com/v1/tracks/6MEtlpTuzFJQ...,https://api.spotify.com/v1/audio-analysis/6MEt...,225830,4
Pa Que Sepa,0.773,0.924,7,-4.249,1,0.1490,0.1270,0.000000,0.352,0.925,105.000,audio_features,4ssX6WT1BA6deuNEIqe8C3,spotify:track:4ssX6WT1BA6deuNEIqe8C3,https://api.spotify.com/v1/tracks/4ssX6WT1BA6d...,https://api.spotify.com/v1/audio-analysis/4ssX...,195907,4
Scherzo,0.504,0.158,4,-16.871,1,0.0399,0.9840,0.466000,0.139,0.683,131.614,audio_features,1KUFkemmTNsq4qvnbofSeW,spotify:track:1KUFkemmTNsq4qvnbofSeW,https://api.spotify.com/v1/tracks/1KUFkemmTNsq...,https://api.spotify.com/v1/audio-analysis/1KUF...,200264,4


In [13]:
candidatos_df = candidatos_df[["id", "acousticness", "danceability", "duration_ms", "energy", "instrumentalness",  "key", "liveness", "loudness", "mode", "speechiness", "tempo", "valence"]]

In [14]:
candidatos_df.shape

(743, 13)

In [15]:
candidatos_df.head()

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
Aguije,5jMW9bjIpyziEv5mu5Next,0.2080,0.467,191332,0.871,0.000000,0,0.456,-5.292,1,0.2180,150.013,0.605
La Distancia,2MdaSGTSLkY2ZEVYTEFjAl,0.0876,0.685,228548,0.687,0.000006,0,0.078,-4.580,1,0.0370,159.993,0.949
La esquina de Rowland,6MEtlpTuzFJQfa8DvZ6N4V,0.1530,0.694,225830,0.780,0.000000,0,0.147,-4.573,1,0.0329,129.919,0.635
Pa Que Sepa,4ssX6WT1BA6deuNEIqe8C3,0.1270,0.773,195907,0.924,0.000000,7,0.352,-4.249,1,0.1490,105.000,0.925
Scherzo,1KUFkemmTNsq4qvnbofSeW,0.9840,0.504,200264,0.158,0.466000,4,0.139,-16.871,1,0.0399,131.614,0.683


In [16]:
toptracks_df.head()

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
Pedindo Esmolas,2ETyUK0F39pIYkkoQu6lRu,0.8120,0.779,220721,0.287,0.000047,2,0.1700,-14.267,0,0.0342,117.982,0.697
Volví,2vmfvSoZBFAt9hhRoEByLi,0.4220,0.725,230125,0.860,0.000284,1,0.0674,-3.726,1,0.1820,175.896,0.791
Promise (feat. Usher),0jr6tT2vc4cIFPHb6wufG3,0.4030,0.764,252253,0.739,0.000000,1,0.0821,-6.422,1,0.0574,132.995,0.770
De Ellos Aprendí,1DnQiHRa2XnfwIf8qVl5C1,0.1450,0.720,266076,0.572,0.000000,7,0.4390,-6.479,1,0.1160,166.843,0.675
Me Voy Enamorando,39Gb80bu5xF0rQtAxTxUS1,0.0538,0.685,235720,0.912,0.000000,10,0.3920,-3.411,1,0.0687,99.957,0.511


In [18]:
# Guardar los resultados en un archivo csv
candidatos_df.to_csv('data/default_candidatos.csv')
toptracks_df.to_csv('data/default_toptracks.csv')

In [19]:
candidatos_df = pd.read_csv('data/default_candidatos.csv')
toptracks_df = pd.read_csv('data/default_toptracks.csv')

In [20]:
candidatos_df.head()

,Unnamed: 0,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
0,Aguije,5jMW9bjIpyziEv5mu5Next,0.2080,0.467,191332,0.871,0.000000,0,0.456,-5.292,1,0.2180,150.013,0.605
1,La Distancia,2MdaSGTSLkY2ZEVYTEFjAl,0.0876,0.685,228548,0.687,0.000006,0,0.078,-4.580,1,0.0370,159.993,0.949
2,La esquina de Rowland,6MEtlpTuzFJQfa8DvZ6N4V,0.1530,0.694,225830,0.780,0.000000,0,0.147,-4.573,1,0.0329,129.919,0.635
3,Pa Que Sepa,4ssX6WT1BA6deuNEIqe8C3,0.1270,0.773,195907,0.924,0.000000,7,0.352,-4.249,1,0.1490,105.000,0.925
4,Scherzo,1KUFkemmTNsq4qvnbofSeW,0.9840,0.504,200264,0.158,0.466000,4,0.139,-16.871,1,0.0399,131.614,0.683


In [21]:
toptracks_df.head()

,Unnamed: 0,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
0,Pedindo Esmolas,2ETyUK0F39pIYkkoQu6lRu,0.8120,0.779,220721,0.287,0.000047,2,0.1700,-14.267,0,0.0342,117.982,0.697
1,Volví,2vmfvSoZBFAt9hhRoEByLi,0.4220,0.725,230125,0.860,0.000284,1,0.0674,-3.726,1,0.1820,175.896,0.791
2,Promise (feat. Usher),0jr6tT2vc4cIFPHb6wufG3,0.4030,0.764,252253,0.739,0.000000,1,0.0821,-6.422,1,0.0574,132.995,0.770
3,De Ellos Aprendí,1DnQiHRa2XnfwIf8qVl5C1,0.1450,0.720,266076,0.572,0.000000,7,0.4390,-6.479,1,0.1160,166.843,0.675
4,Me Voy Enamorando,39Gb80bu5xF0rQtAxTxUS1,0.0538,0.685,235720,0.912,0.000000,10,0.3920,-3.411,1,0.0687,99.957,0.511
